# **A11 - Ensemble Models**
Mitch Messier, December 7, 2023

# Table of Contents

1. **Task 1:** Set up, Data import, and Preparation
2. **Task 2:** Simple Model
3. **Task 3:** 3 Good models whose best hyperparameters were discovered using gridsearch
4. **Task 4:** Voting Ensemble Classifier
5. **Task 5:** Boosting Model
6. **Task 6:** Stacked Ensemble Classifier

# Load Libraries

In [216]:
import pandas as pd
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

# **Task 1:** Set up, Data import, and Preparation

In [167]:
url = "https://raw.githubusercontent.com/matthewpecsok/4482_fall_2022/main/data/CD_additional_modified.csv"
data = pd.read_csv(url)

In [168]:
print("Data Frame Structure:")
data.info()

Data Frame Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4117 non-null   int64  
 1   job             4117 non-null   object 
 2   marital         4117 non-null   object 
 3   education       4117 non-null   object 
 4   default         4117 non-null   object 
 5   housing         4117 non-null   object 
 6   loan            4117 non-null   object 
 7   contact         4117 non-null   object 
 8   month           4117 non-null   object 
 9   day_of_week     4117 non-null   object 
 10  duration        4117 non-null   int64  
 11  campaign        4117 non-null   int64  
 12  pdays           4117 non-null   int64  
 13  previous        4117 non-null   int64  
 14  poutcome        4117 non-null   object 
 15  emp_var_rate    4117 non-null   float64
 16  cons_price_idx  4117 non-null   float64
 17  cons_conf_i

In [169]:
print("\nData Frame Summary:")
data.describe()


Data Frame Summary:
               age     duration     campaign        pdays     previous  \
count  4117.000000  4117.000000  4117.000000  4117.000000  4117.000000   
mean     40.115375   256.850376     2.537042   960.403449     0.190187   
std      10.314847   254.749615     2.568668   191.967524     0.541765   
min      18.000000     0.000000     1.000000     0.000000     0.000000   
25%      32.000000   103.000000     1.000000   999.000000     0.000000   
50%      38.000000   181.000000     2.000000   999.000000     0.000000   
75%      47.000000   317.000000     3.000000   999.000000     0.000000   
max      88.000000  3643.000000    35.000000   999.000000     6.000000   

       emp_var_rate  cons_price_idx  cons_conf_idx    euribor3m  nr_employed  
count   4117.000000     4117.000000    4117.000000  4117.000000  4117.000000  
mean       0.085742       93.580131     -40.500947     3.621904  5166.496502  
std        1.562799        0.579061       4.593445     1.733448    73.67094

In [170]:
print("\nFirst Few Rows of the Data Frame:")
data.head()


First Few Rows of the Data Frame:
   age          job  marital          education default  housing     loan  \
0   30  blue-collar  married           basic.9y      no      yes       no   
1   39     services   single        high.school      no       no       no   
2   25     services  married        high.school      no      yes       no   
3   38     services  married           basic.9y      no  unknown  unknown   
4   47       admin.  married  university.degree      no      yes       no   

     contact month day_of_week  duration  campaign  pdays  previous  \
0   cellular   may         fri       487         2    999         0   
1  telephone   may         fri       346         4    999         0   
2  telephone   jun         wed       227         1    999         0   
3  telephone   jun         fri        17         3    999         0   
4   cellular   nov         mon        58         1    999         0   

      poutcome  emp_var_rate  cons_price_idx  cons_conf_idx  euribor3m  \
0

In [171]:
y_target = data.pop('y')

In [172]:
y_target = y_target.eq('yes').mul(1)

In [173]:
print("Data Frame Structure after popping target variable:")
data.info()

Data Frame Structure after popping target variable:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4117 non-null   int64  
 1   job             4117 non-null   object 
 2   marital         4117 non-null   object 
 3   education       4117 non-null   object 
 4   default         4117 non-null   object 
 5   housing         4117 non-null   object 
 6   loan            4117 non-null   object 
 7   contact         4117 non-null   object 
 8   month           4117 non-null   object 
 9   day_of_week     4117 non-null   object 
 10  duration        4117 non-null   int64  
 11  campaign        4117 non-null   int64  
 12  pdays           4117 non-null   int64  
 13  previous        4117 non-null   int64  
 14  poutcome        4117 non-null   object 
 15  emp_var_rate    4117 non-null   float64
 16  cons_price_idx  4117 non-n

In [174]:
print("\nData Frame Summary after popping target variable:")
data.describe()


Data Frame Summary after popping target variable:


,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000
mean,40.115375,256.850376,2.537042,960.403449,0.190187,0.085742,93.580131,-40.500947,3.621904,5166.496502
std,10.314847,254.749615,2.568668,191.967524,0.541765,1.562799,0.579061,4.593445,1.733448,73.670942
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.635000,4963.600000
25%,32.000000,103.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000,5099.100000
50%,38.000000,181.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.000000,317.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,88.000000,3643.000000,35.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [175]:
print("\nFirst Few Rows of the Data Frame after popping target variable:")
data.head()


First Few Rows of the Data Frame after popping target variable:


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,487,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1
1,39,services,single,high.school,no,no,no,telephone,may,fri,346,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,227,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,17,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,58,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8


Dummy encoding is necessary for this dataset as it contains categorical variables. These variables need to be converted into a format that can be understood by machine learning algorithms, which typically work better with numerical input.


In [176]:
df_encoded = pd.get_dummies(data)

In [177]:
print("Encoded Data Frame Structure:")
df_encoded.info()

Encoded Data Frame Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            4117 non-null   int64  
 1   duration                       4117 non-null   int64  
 2   campaign                       4117 non-null   int64  
 3   pdays                          4117 non-null   int64  
 4   previous                       4117 non-null   int64  
 5   emp_var_rate                   4117 non-null   float64
 6   cons_price_idx                 4117 non-null   float64
 7   cons_conf_idx                  4117 non-null   float64
 8   euribor3m                      4117 non-null   float64
 9   nr_employed                    4117 non-null   float64
 10  job_admin.                     4117 non-null   uint8  
 11  job_blue-collar                4117 non-null   uint8  
 12  job_entrepreneur  

In [178]:
print("\nEncoded Data Frame Summary:")
df_encoded.describe()


Encoded Data Frame Summary:


,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
count,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000,4117.000000
mean,40.115375,256.850376,2.537042,960.403449,0.190187,0.085742,93.580131,-40.500947,3.621904,5166.496502,0.245810,0.214719,0.035949,0.026718,0.078698,0.040078,0.038620,0.095458,0.019917,0.167841,0.026718,0.009473,0.108088,0.609181,0.280058,0.002672,0.104202,0.055380,0.139422,0.223464,0.129949,0.307020,0.040564,0.804955,0.195045,0.446199,0.025504,0.528297,0.812971,0.025504,0.161525,0.643673,0.356327,0.052222,0.154239,0.005344,0.172699,0.128735,0.011659,0.334710,0.108088,0.016760,0.015545,0.186544,0.207675,0.208890,0.204032,0.192859,0.110032,0.855477,0.034491
std,10.314847,254.749615,2.568668,191.967524,0.541765,1.562799,0.579061,4.593445,1.733448,73.670942,0.430619,0.410677,0.186184,0.161279,0.269300,0.196165,0.192712,0.293882,0.139734,0.373770,0.161279,0.096879,0.310530,0.487993,0.449082,0.051627,0.305560,0.228749,0.346428,0.416617,0.336288,0.461314,0.197301,0.396283,0.396283,0.497157,0.157669,0.499259,0.389982,0.157669,0.368060,0.478972,0.478972,0.222502,0.361221,0.072914,0.378032,0.334947,0.107358,0.471946,0.310530,0.128386,0.123723,0.389592,0.405692,0.406565,0.403041,0.394591,0.312967,0.351662,0.182509
min,18.000000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.635000,4963.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32.000000,103.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000,5099.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,38.000000,181.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [179]:
print("\nFirst Few Rows of the Encoded Data Frame:")
df_encoded.head()


First Few Rows of the Encoded Data Frame:


,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,30,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
1,39,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
2,25,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
3,38,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
4,47,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0


# **Task 2:** Simple Model

In [181]:
model = DecisionTreeClassifier()

In [182]:
cv_results = cross_validate(model, df_encoded, y_target, cv=5, scoring='f1', return_train_score=True)

In [183]:
cv_results_df = pd.DataFrame(cv_results)

In [184]:
mean_scores = cv_results_df.mean()
std_scores = cv_results_df.std()

In [187]:
print("Cross Validation F1 Scores:")
print(cv_results_df['test_score'])
print("\nMean F1 Score:", mean_scores['test_score'])
print("Standard Deviation of F1 Score:", std_scores['test_score'])

Cross Validation F1 Scores:
0    0.469274
1    0.443243
2    0.531915
3    0.416185
4    0.505263
Name: test_score, dtype: float64

Mean F1 Score: 0.47317600177400543
Standard Deviation of F1 Score: 0.04647315955746442


# **Task 3:** 3 Good models whose best hyperparameters were discovered using gridsearch

## RandomForest Model

In [189]:
param_grid_rf = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40]
}

In [190]:
rf_model = RandomForestClassifier()
grid_search_rf = GridSearchCV(rf_model, param_grid_rf, cv=5, scoring='f1', return_train_score=True)

In [191]:
grid_search_rf.fit(df_encoded, y_target)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30, 40],
                         'n_estimators': [10, 50, 100, 200]},
             return_train_score=True, scoring='f1')

In [193]:
best_rf = grid_search_rf.best_estimator_
best_rf_results = grid_search_rf.cv_results_
best_rf_index = grid_search_rf.best_index_

In [194]:
print("Best RandomForest Model Mean F1 Score:", best_rf_results['mean_test_score'][best_rf_index])
print("Standard Deviation:", best_rf_results['std_test_score'][best_rf_index])
print("Best Parameters:", best_rf.get_params())

Best RandomForest Model Mean F1 Score: 0.4611410539824445
Standard Deviation: 0.018360367177122583
Best Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


##  SVC Model with Sigmoid Kernel

In [196]:
param_grid_svc_sigmoid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['sigmoid']
}

In [197]:
svc_sigmoid_model = SVC()
grid_search_svc_sigmoid = GridSearchCV(svc_sigmoid_model, param_grid_svc_sigmoid, cv=5, scoring='f1', return_train_score=True)

In [198]:
grid_search_svc_sigmoid.fit(df_encoded, y_target)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'kernel': ['sigmoid']},
             return_train_score=True, scoring='f1')

In [199]:
best_svc_sigmoid = grid_search_svc_sigmoid.best_estimator_
best_svc_sigmoid_results = grid_search_svc_sigmoid.cv_results_
best_svc_sigmoid_index = grid_search_svc_sigmoid.best_index_

In [200]:
print("Best SVC Sigmoid Model Mean F1 Score:", best_svc_sigmoid_results['mean_test_score'][best_svc_sigmoid_index])
print("Standard Deviation:", best_svc_sigmoid_results['std_test_score'][best_svc_sigmoid_index])
print("Best Parameters:", best_svc_sigmoid.get_params())

Best SVC Sigmoid Model Mean F1 Score: 0.3854819208093475
Standard Deviation: 0.05452723128332802
Best Parameters: {'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'sigmoid', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


## SVC Model with Radial Basis Kernel

In [201]:
param_grid_svc_rbf = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf']
}

In [202]:
svc_rbf_model = SVC()
grid_search_svc_rbf = GridSearchCV(svc_rbf_model, param_grid_svc_rbf, cv=5, scoring='f1', return_train_score=True)

In [203]:
grid_search_svc_rbf.fit(df_encoded, y_target)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'kernel': ['rbf']},
             return_train_score=True, scoring='f1')

In [204]:
best_svc_rbf = grid_search_svc_rbf.best_estimator_
best_svc_rbf_results = grid_search_svc_rbf.cv_results_
best_svc_rbf_index = grid_search_svc_rbf.best_index_

In [205]:
print("Best SVC Radial Basis Model Mean F1 Score:", best_svc_rbf_results['mean_test_score'][best_svc_rbf_index])
print("Standard Deviation:", best_svc_rbf_results['std_test_score'][best_svc_rbf_index])
print("Best Parameters:", best_svc_rbf.get_params())

Best SVC Radial Basis Model Mean F1 Score: 0.3741742505777132
Standard Deviation: 0.06776840054202035
Best Parameters: {'C': 100, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


# **Task 4:** Voting Ensemble Classifier

## Create the Models with Best Parameters

In [207]:
best_rf_model = RandomForestClassifier(**grid_search_rf.best_params_)
best_svc_sigmoid_model = SVC(**grid_search_svc_sigmoid.best_params_)
best_svc_rbf_model = SVC(**grid_search_svc_rbf.best_params_)

## Create VotingClassifier

In [208]:
voting_clf = VotingClassifier(
    estimators=[('rf', best_rf_model), ('svc_sigmoid', best_svc_sigmoid_model), ('svc_rbf', best_svc_rbf_model)],
    voting='hard'
)

## Evaluate the VotingClassifier

In [209]:
voting_cv_results = cross_validate(voting_clf, df_encoded, y_target, cv=5, scoring='f1')

In [210]:
voting_cv_results_df = pd.DataFrame(voting_cv_results)
mean_voting_f1_score = voting_cv_results_df['test_score'].mean()

In [211]:
print("Mean F1 Score for Voting Classifier:", mean_voting_f1_score)

Mean F1 Score for Voting Classifier: 0.37730721799309974


The Voting Ensemble method combines different machine learning classifiers and uses a majority vote (hard voting) or the average predicted probabilities (soft voting) to predict the class labels. Such an ensemble model often achieves higher performance compared to a single model due to the different strengths and perspectives of the individual models. In hard voting, the predicted output class is a class with the majority of votes, whereas in soft voting, the output class is the prediction based on the average of probabilities given to that class.

# **Task 5:** Boosting Model

## Create and Evaluate GradientBoostingClassifier using GridSearchCV

In [213]:
param_grid_gbc = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

In [214]:
gbc = GradientBoostingClassifier()
grid_search_gbc = GridSearchCV(gbc, param_grid_gbc, cv=5, scoring='f1', return_train_score=True)

In [215]:
grid_search_gbc.fit(df_encoded, y_target)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='f1')

In [217]:
gbc_results_df = pd.DataFrame(grid_search_gbc.cv_results_)
best_gbc_result = gbc_results_df[gbc_results_df['rank_test_score'] == 1]

In [218]:
print("Best Gradient Boosting Classifier Result:")
print(best_gbc_result[['mean_test_score', 'std_test_score', 'params']])

Best Gradient Boosting Classifier Result:
    mean_test_score  std_test_score  \
11         0.562446        0.033639   

                                               params  
11  {'learning_rate': 0.1, 'max_depth': 3, 'n_esti...  


##  Description of Boosting Model

Boosting is a machine learning ensemble meta-algorithm that is primarily used to reduce bias and variance in supervised learning. It works by combining multiple weak learners into a strong learner in a sequential manner. Each model attempts to correct the errors made by the previous one. In the context of gradient boosting, the model fits new predictors to the residual errors made by previous predictors and then combines these predictors through a weighted majority vote (or sum) to make the final prediction. This approach allows the model to give more focus to the more difficult cases, improving the model's performance and accuracy.

# **Task 6:** Stacked Ensemble Classifier

## Create Stacked Ensemble Classifier

In [219]:
best_rf_model = RandomForestClassifier(**grid_search_rf.best_params_)
best_svc_sigmoid_model = SVC(**grid_search_svc_sigmoid.best_params_, probability=True)
best_svc_rbf_model = SVC(**grid_search_svc_rbf.best_params_, probability=True)

In [220]:
stacked_clf = StackingClassifier(
    estimators=[('rf', best_rf_model), ('svc_sigmoid', best_svc_sigmoid_model), ('svc_rbf', best_svc_rbf_model)],
    final_estimator=GradientBoostingClassifier()
)

## Evaluate the StackingClassifier

In [221]:
stacking_cv_results = cross_validate(stacked_clf, df_encoded, y_target, cv=5, scoring='f1')

In [222]:
stacking_cv_results_df = pd.DataFrame(stacking_cv_results)
mean_stacking_f1_score = stacking_cv_results_df['test_score'].mean()

In [223]:
print("Mean F1 Score for Stacking Classifier:", mean_stacking_f1_score)

Mean F1 Score for Stacking Classifier: 0.5653994018633159


## Description of Stacked Ensemble Method

Stacked Ensemble is an ensemble learning technique that combines multiple classification or regression models via a meta-classifier or a meta-regressor. The individual models are trained based on the complete training set; then, the meta-model is trained on the outputs of the individual models as features. This approach can be viewed as a way of combining the strengths of various models to improve prediction performance. The stacking method allows the ensemble to learn how best to combine the predictions from the contributing models, rather than relying on simple techniques like voting or averaging.

# Convert to HTML

In [224]:
# Insert a code cell to include to following line of command in each
# Colab notebook to access data files saved in Google Drive of your account
# Retrieve csv file from google drive by mapping the folder from google drive.
# Must be done each time session expires.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [225]:
# Copy the ipynb to the local working directory
# Replace 'A7_Messier_Mitch.ipynb' with your actual file name if it's different
!cp "/content/drive/MyDrive/Colab Notebooks/A11_Messier_Mitch.ipynb" ./

# Create an HTML file from the ipynb
!jupyter nbconvert --to html "A11_Messier_Mitch.ipynb"

[NbConvertApp] Converting notebook A11_Messier_Mitch.ipynb to html
[NbConvertApp] Writing 742425 bytes to A11_Messier_Mitch.html
